## Importing all the libraries

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
keras = tf.keras
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go
physical_devices = tf.config.experimental.list_physical_devices('GPU')

## Helper Functions

In [28]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

## Importing CoronaVirus Cases Data

In [29]:
confirmed_df = pd.read_csv('COVID-19-master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
death_df = pd.read_csv('COVID-19-master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recovered_df = pd.read_csv('COVID-19-master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

## Data Pre Processing

### Confirmed Data

In [30]:
confirmed_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,209322,209340,209358,209362,209369,209390,209406,209436,209451,209451
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,334391,334408,334408,334427,334427,334427,334427,334427,334443,334457
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,271441,271448,271463,271469,271469,271477,271477,271490,271494,271496
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,47866,47875,47875,47875,47875,47875,47875,47875,47890,47890
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,105255,105277,105277,105277,105277,105277,105277,105277,105288,105288


In [31]:
death_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,7896,7896,7896,7896,7896,7896,7896,7896,7896,7896
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3598,3598,3598,3598,3598,3598,3598,3598,3598,3598
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,6881,6881,6881,6881,6881,6881,6881,6881,6881,6881
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,165,165,165,165,165,165,165,165,165,165
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,1933,1933,1933,1933,1933,1933,1933,1933,1933,1933


In [32]:
recovered_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
confirmed_df = confirmed_df.iloc[:, 4:]
confirmed_df = confirmed_df.sum(axis=0)
confirmed_df.index = pd.to_datetime(confirmed_df.index).strftime('%Y-%m-%d')
confirmed_df = pd.DataFrame(confirmed_df)

C:\Users\Siddharth\AppData\Local\Temp\ipykernel_16432\2800806791.py:3: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



### Death Data

In [34]:
death_df = death_df.iloc[:, 4:]
death_df = death_df.sum(axis=0)
death_df.index = pd.to_datetime(death_df.index).strftime('%Y-%m-%d')
death_df = pd.DataFrame(death_df)

C:\Users\Siddharth\AppData\Local\Temp\ipykernel_16432\2288649238.py:3: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



### Recovered Data

In [35]:
recovered_df = recovered_df.iloc[:, 4:]
recovered_df = recovered_df.sum(axis=0)
recovered_df.index = pd.to_datetime(recovered_df.index).strftime('%Y-%m-%d')
recovered_df = pd.DataFrame(recovered_df)

C:\Users\Siddharth\AppData\Local\Temp\ipykernel_16432\1784248634.py:3: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



# Overall Corona Cases

# Scaling Data

### Confirmed Cases Data

In [36]:
scaler_confirm = MinMaxScaler()
scaler_confirm = scaler_confirm.fit(confirmed_df)
train_confirm = scaler_confirm.fit_transform(confirmed_df)
train_confirm = pd.DataFrame(train_confirm)

### Death Cases Data

In [37]:
scaler_death = MinMaxScaler()
scaler_death = scaler_death.fit(death_df)
train_death = scaler_death.transform(death_df)
train_death = pd.DataFrame(train_death)

### Recovered Data

In [38]:
scaler_recovered = MinMaxScaler()
scaler_recovered = scaler_recovered.fit(recovered_df)
train_recovered = scaler_recovered.fit_transform(recovered_df)
train_recovered = pd.DataFrame(train_recovered)

## Preparing Data For Time Series Analysis Using LSTM

In [39]:
time_steps = 1
X_train_confirm, y_train_confirm = create_dataset(train_confirm, train_confirm, time_steps)
X_train_death, y_train_death = create_dataset(train_death, train_death, time_steps)
X_train_recovered, y_train_recovered = create_dataset(train_recovered, train_recovered, time_steps)

## Model Building

## Model for Confirmed Cases

In [40]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

confirm_model = keras.Sequential()
confirm_model.add(keras.layers.LSTM(units=128,input_shape=(X_train_confirm.shape[1], X_train_confirm.shape[2]),return_sequences = True))
confirm_model.add(keras.layers.LSTM(units=128,return_sequences = True))
confirm_model.add(keras.layers.LSTM(units=128,return_sequences = True))
confirm_model.add(keras.layers.LSTM(units=128))
confirm_model.add(keras.layers.Dropout(rate=0.2))
confirm_model.add(keras.layers.Dense(units=1))
confirm_model.compile(loss='mean_squared_error', optimizer='adam') 
reset_states = ResetStatesCallback()
confirm_model.fit(X_train_confirm, y_train_confirm,epochs=500,batch_size=16,shuffle=False,callbacks = [reset_states],verbose = 0)

## Model for Death Cases

In [41]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

death_model = keras.Sequential()
death_model.add(keras.layers.LSTM(units=128,input_shape=(X_train_death.shape[1], X_train_death.shape[2]),return_sequences = True))
death_model.add(keras.layers.LSTM(units=128,return_sequences = True))
death_model.add(keras.layers.LSTM(units=128,return_sequences = True))
death_model.add(keras.layers.LSTM(units=128))
death_model.add(keras.layers.Dropout(rate=0.2))
death_model.add(keras.layers.Dense(units=1))
death_model.compile(loss='mean_squared_error', optimizer='adam') 
reset_states = ResetStatesCallback()
death_model.fit(X_train_death, y_train_death,epochs=500,batch_size=16,shuffle=False,callbacks = [reset_states],verbose=0)


## Model for Recovered Cases

In [42]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

recovered_model = keras.Sequential()
recovered_model.add(keras.layers.LSTM(units=128,input_shape=(X_train_recovered.shape[1], X_train_recovered.shape[2]),return_sequences = True))
recovered_model.add(keras.layers.LSTM(units=128,return_sequences = True))
recovered_model.add(keras.layers.LSTM(units=128,return_sequences = True))
recovered_model.add(keras.layers.LSTM(units=128))
recovered_model.add(keras.layers.Dropout(rate=0.2))
recovered_model.add(keras.layers.Dense(units=1))
recovered_model.compile(loss='mean_squared_error', optimizer='adam') 
reset_states = ResetStatesCallback()
recovered_model.fit(X_train_recovered, y_train_recovered,epochs=500,batch_size=16,shuffle=False,callbacks = [reset_states],verbose=0)


## Predicting The Results

In [43]:
DAYS_TO_PREDICT = int(input('Enter Number Of Days You Want to Predict : '))

### Predicting Results For Confirmed Cases

In [44]:
confirm_test_seq  = y_train_confirm[-1:]
confirm_preds = []
for _ in range(DAYS_TO_PREDICT):
    y2_confirm = confirm_test_seq.reshape((len(confirm_test_seq ), time_steps, 1))
    confirm_pred = confirm_model.predict(y2_confirm)
    confirm_preds.append(confirm_pred)
    confirm_new_seq = confirm_test_seq.flatten()
    confirm_new_seq = np.append(confirm_new_seq,[confirm_pred])
    confirm_new_seq = confirm_new_seq[-1:]
    confirm_test_seq = confirm_new_seq.reshape((len(confirm_new_seq), time_steps, 1))
    y2_confirm = confirm_test_seq
    
confirm_preds = (np.array(confirm_preds).flatten()).reshape(-1,1)
confirm_preds = scaler_confirm.inverse_transform(confirm_preds)

predicted_confirmed_index = pd.date_range(start=confirmed_df.index[-1],periods=DAYS_TO_PREDICT+1,inclusive='right').strftime('%Y-%m-%d')
predicted_confirmed_cases = pd.DataFrame()
predicted_confirmed_cases['Dates'] = predicted_confirmed_index
predicted_confirmed_cases['Confirmed Cases Predictions'] = confirm_preds
predicted_confirmed_cases.set_index('Dates', inplace = True)

1/1 [==============================] - 0s 70ms/step


In [45]:
fig = go.Figure()
fig.update_layout(template='plotly_dark')
fig.update_xaxes(tickangle=90, showticklabels = True, type = 'category')
fig.add_trace(go.Scatter(x=predicted_confirmed_cases.index, 
                         y=predicted_confirmed_cases['Confirmed Cases Predictions'],
                         mode='lines+markers',
                         name='confirm',
                         line=dict(color='red', width=2)))

### Predicting Results For Death Cases

In [46]:
death_test_seq  = y_train_death[-1:]
death_preds = []
for i in range(DAYS_TO_PREDICT):
    y2_death = death_test_seq.reshape((len(death_test_seq), time_steps, 1))
    death_pred = death_model.predict(y2_death)
    death_preds.append(death_pred)
    death_new_seq = death_test_seq.flatten()
    death_new_seq = np.append(death_new_seq,[death_pred])
    death_new_seq = death_new_seq[-1:]
    death_test_seq = death_new_seq.reshape((len(death_new_seq), time_steps, 1))
    y2_death = death_test_seq
    
death_preds = (np.array(death_preds).flatten()).reshape(-1,1)
death_preds = scaler_death.inverse_transform(death_preds)

predict_death_index = pd.date_range(start=death_df.index[-1],periods=DAYS_TO_PREDICT + 1,inclusive='right').strftime('%Y-%m-%d')
predict_death_cases = pd.DataFrame()
predict_death_cases['Dates'] = predict_death_index
predict_death_cases['Death Cases Predictions'] = death_preds
predict_death_cases.set_index('Dates', inplace = True)

1/1 [==============================] - 0s 98ms/step


In [47]:
fig = go.Figure()
fig.update_layout(template='plotly_dark')
fig.update_xaxes(tickangle=90, showticklabels = True, type = 'category')
fig.add_trace(go.Scatter(x=predict_death_cases.index, 
                         y=predict_death_cases['Death Cases Predictions'],
                         mode='lines+markers',
                         name='death',
                         line=dict(color='blue', width=2)))

### Predicting Results For Recovered Cases

In [48]:
recovered_test_seq  = y_train_recovered[-1:]
recovered_preds = []
for i in range(DAYS_TO_PREDICT):
    y2_recovered = recovered_test_seq.reshape((len(recovered_test_seq), time_steps, 1))
    recovered_pred = recovered_model.predict(y2_recovered)
    recovered_preds.append(recovered_pred)
    recovered_new_seq = recovered_test_seq.flatten()
    recovered_new_seq = np.append(recovered_new_seq,[recovered_pred])
    recovered_new_seq = recovered_new_seq[-1:]
    recovered_test_seq = recovered_new_seq.reshape((len(recovered_new_seq), time_steps, 1))
    y2_recovered = recovered_test_seq
    
recovered_preds = (np.array(recovered_preds).flatten()).reshape(-1,1)
recovered_preds = scaler_recovered.inverse_transform(recovered_preds)

predict_recovered_index = pd.date_range(start=recovered_df.index[-1],periods=DAYS_TO_PREDICT + 1,inclusive='right').strftime('%Y-%m-%d')

predict_recovered_cases = pd.DataFrame()
predict_recovered_cases['Dates'] = predict_recovered_index
predict_recovered_cases['Recovered Cases Predictions'] = recovered_preds
predict_recovered_cases.set_index('Dates', inplace = True)

1/1 [==============================] - 0s 57ms/step


In [49]:
fig = go.Figure()
fig.update_layout(template='plotly_dark')
fig.update_xaxes(tickangle=90, showticklabels = True, type = 'category')
fig.add_trace(go.Scatter(x=predict_recovered_cases.index, 
                         y=predict_recovered_cases['Recovered Cases Predictions'],
                         mode='lines+markers',
                         name='recovered',
                         line=dict(color='green', width=2)))

# Visualizations

### Historial Cases

In [50]:
fig = go.Figure()
fig.update_layout(template='plotly_dark')
fig.update_xaxes(tickangle=90, showticklabels = True, type = 'category')
fig.add_trace(go.Scatter(x=confirmed_df.index, 
                         y=confirmed_df[0],
                         mode='lines+markers',
                         name='confirm',
                         line=dict(color='red', width=2)))
fig.add_trace(go.Scatter(x=death_df.index, 
                         y=death_df[0],
                         mode='lines+markers',
                         name='death',
                         line=dict(color='green', width=2)))
fig.add_trace(go.Scatter(x=recovered_df.index, 
                         y=recovered_df[0],
                         mode='lines+markers',
                         name='recovered',
                         line=dict(color='blue', width=2)))

## Predictions for every individual scenario

In [51]:
fig = go.Figure()
fig.update_layout(template='plotly_dark')
fig.update_xaxes(tickangle=90, showticklabels = True, type = 'category')
fig.add_trace(go.Scatter(x=predicted_confirmed_cases.index, 
                         y=predicted_confirmed_cases['Confirmed Cases Predictions'],
                         mode='lines+markers',
                         name='Confirmed Cases Predictions',
                         line=dict(color='yellow', width=2)))

fig.add_trace(go.Scatter(x=predict_death_cases.index, 
                         y=predict_death_cases['Death Cases Predictions'],
                         mode='lines+markers',
                         name='Death Cases Predictions',
                         line=dict(color='turquoise', width=2)))

fig.add_trace(go.Scatter(x=predict_recovered_cases.index, 
                         y=predict_recovered_cases['Recovered Cases Predictions'],
                         mode='lines+markers',
                         name='Recovered Cases Predictions',
                         line=dict(color='white', width=2)))

## Historical and Predicted Values

In [52]:
fig = go.Figure()
fig.update_layout(template='plotly_dark')
fig.update_xaxes(tickangle=90, showticklabels = True, type = 'category')
fig.add_trace(go.Scatter(x=confirmed_df.index, 
                         y=confirmed_df[0],
                         mode='lines+markers',
                         name='confirm',
                         line=dict(color='red', width=2)))

fig.add_trace(go.Scatter(x=death_df.index, 
                         y=death_df[0],
                         mode='lines+markers',
                         name='death',
                         line=dict(color='green', width=2)))

fig.add_trace(go.Scatter(x=recovered_df.index, 
                         y=recovered_df[0],
                         mode='lines+markers',
                         name='recovered',
                         line=dict(color='blue', width=2)))

fig.add_trace(go.Scatter(x=predicted_confirmed_cases.index, 
                         y=predicted_confirmed_cases['Confirmed Cases Predictions'],
                         mode='lines+markers',
                         name='Confirmed Cases Predictions',
                         line=dict(color='yellow', width=2)))

fig.add_trace(go.Scatter(x=predict_death_cases.index, 
                         y=predict_death_cases['Death Cases Predictions'],
                         mode='lines+markers',
                         name='Death Cases Predictions',
                         line=dict(color='turquoise', width=2)))

fig.add_trace(go.Scatter(x=predict_recovered_cases.index, 
                         y=predict_recovered_cases['Recovered Cases Predictions'],
                         mode='lines+markers',
                         name='Recovered Cases Predictions',
                         line=dict(color='white', width=2)))
